# Experts RS Main
This note book is for running our ExpertsRS prototype system


## Set up the API
Set up your LLM api in EXPERTS_RS_CONFIG_LIST

In [ ]:
import autogen
from prompts import scientist_prompt, engineer_prompt, manager_prompt, user_proxy_prompt, executor_prompt

config_list = autogen.config_list_from_json("EXPERTS_RS_CONFIG_LIST") # set the api key in the config list
print("LLM models: ", [config_list[i]["model"] for i in range(len(config_list))]) 

In [ ]:
# llm config
gpt4_config = {
    "cache_seed": 9,  # change the cache_seed for different trials
    "temperature": 0,
    "config_list": config_list,
    "timeout": 120, # in seconds
}

## Construct Agents
Construct agents

In [ ]:
# The User proxy
user_proxy = autogen.UserProxyAgent(
    name="User",
    system_message= user_proxy_prompt,
    code_execution_config=False,
    human_input_mode="ALWAYS",
)

# The Manager
manager = autogen.AssistantAgent(
    name="Manager",
    system_message= manager_prompt,
    llm_config=gpt4_config
)

# The Scientist
scientist = autogen.AssistantAgent(
    name="Scientist",
    system_message= scientist_prompt,
    llm_config=gpt4_config
)

# The Engineer
engineer = autogen.AssistantAgent(
    name="Engineer",
    system_message=engineer_prompt,
    llm_config=gpt4_config,
    code_execution_config=False,  # Turn off code execution for this agent.

)

# The Executor
executor = autogen.UserProxyAgent(
    name="Executor",
    system_message=executor_prompt,
    human_input_mode="NEVER",  # auto response mode
     code_execution_config={
        "executor": "commandline-local",  # use local environment
        # "work_dir": "/temp",
        "last_n_messages": 3,
    }
)


## Customize agent selction function
Inspired by StateFlow (Wu et al. 2024), a LLM-based task-solving paradigm that conceptualizes complex task-solving processes as state machines.

In [ ]:
# Customized agent selection function
def speaker_selection(last_speaker, groupchat):
    messages = groupchat.messages
    
    if len(messages) <= 1:  
        # Initial state: user starts conversation with Manager
        return manager
    
    if last_speaker == manager:
        # Manager communicates directly with the user
        return user_proxy
    
    if last_speaker == user_proxy:
        # After Manager's communication, if user approves the report
        if "Approve" in messages[-1]["content"]:
            return scientist  # Pass the report to Scientist upon user approval
        elif "End" in messages[-1]["content"]:
            return None  # End the chat if user requests
        return manager  # Continue conversation with Manager if user hasn't approved
    
    # State 2: Transition from Scientist to Engineer for a well-defined research question
    if last_speaker == scientist:
        # Scientist hands over the task to Engineer
        return engineer
    
    if last_speaker == engineer:
        # Engineer generates code and passes it to Executor for execution
        return executor
    
    if last_speaker == executor:
        # Executor executes the code and checks for errors
        if any(keyword in messages[-1]["content"] for keyword in ["error", "failed", "exception"]):
            return engineer  # Return to Engineer for modifications if there are errors
        elif messages[-1]["content"] == "":
            return engineer  # Return to Engineer if output is empty
        else:
            return manager  # Return results to Manager if execution is successful

    return None  # End the conversation if none of the above conditions are met


In [ ]:
# Group chat setting
groupchat = autogen.GroupChat(
    agents=[user_proxy, manager, scientist, engineer, executor], 
    messages=[], 
    max_round=30,
     speaker_selection_method=speaker_selection
)


# chat administrator (group chat manager)
chat_admin = autogen.GroupChatManager(
    name="Chat_Admin",
    llm_config=gpt4_config,
    groupchat = groupchat
)

## Start chatting
Input your request and run this cell

In [ ]:
# initiating chat
users_request = 'I am a researcher in the urban greening department. I want to know the health condition of urban vegetation in my district.' #TODO: enter user's request here

user_proxy.initiate_chat(
    chat_admin,
    message= users_request,
)